<a href="https://colab.research.google.com/github/ClaudiaRuggiero/DM-Project-2020-2021/blob/main/Cancer_classifier_via_TP53gene.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**Path to datasets**

In [ ]:
######## INSERT DATASETS PATH ########

dataset_path = "/content/drive/My Drive/Colab Notebooks/base_changes_dataset_extended.tsv"

biovec_model_path="/content/drive/My Drive/Colab Notebooks/swissprot-reviewed-protvec.model"



#Imports

In [ ]:
import numpy as np

import keras
from keras import models
from keras.layers import Dense, Activation, Dropout, Flatten,\
     Conv1D, MaxPooling1D, LSTM, Bidirectional
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adamax

from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import recall_score, precision_score, accuracy_score
from sklearn.model_selection import train_test_split
from keras.metrics import Precision, Recall, CategoricalAccuracy 

**Biovec package for protein embedding**

In [ ]:
!pip install biovec

#Global vars

In [ ]:
TP53 = ("ATGGAGGAGCCGCAGTCAGATCCTAGCGTCGAGCCCCCTCTGAGTCAGGAAACATTTTCA"\
        "GACCTATGGAAACTACTTCCTGAAAACAACGTTCTGTCCCCCTTGCCGTCCCAAGCAATG"\
        "GATGATTTGATGCTGTCCCCGGACGATATTGAACAATGGTTCACTGAAGACCCAGGTCCA"\
        "GATGAAGCTCCCAGAATGCCAGAGGCTGCTCCCCGCGTGGCCCCTGCACCAGCAGCTCCT"\
        "ACACCGGCGGCCCCTGCACCAGCCCCCTCCTGGCCCCTGTCATCTTCTGTCCCTTCCCAG"\
        "AAAACCTACCAGGGCAGCTACGGTTTCCGTCTGGGCTTCTTGCATTCTGGGACAGCCAAG"\
        "TCTGTGACTTGCACGTACTCCCCTGCCCTCAACAAGATGTTTTGCCAACTGGCCAAGACC"\
        "TGCCCTGTGCAGCTGTGGGTTGATTCCACACCCCCGCCCGGCACCCGCGTCCGCGCCATG"\
        "GCCATCTACAAGCAGTCACAGCACATGACGGAGGTTGTGAGGCGCTGCCCCCACCATGAG"\
        "CGCTGCTCAGATAGCGATGGTCTGGCCCCTCCTCAGCATCTTATCCGAGTGGAAGGAAAT"\
        "TTGCGTGTGGAGTATTTGGATGACAGAAACACTTTTCGACATAGTGTGTGTGTGCCCTAT"\
        "GAGCCGCCTGAGGTTGGCTCTGACTGTACCACCATCCACTACAACTACATGTGTAACAGT"\
        "TCCTGCATGGGCGGCATGAACCGGAGGCCCATCCTCACCATCATCACACTGGAAGACTCC"\
        "AGTGGTAATCTACTGGGACGGAACAGCTTTGAGGTGCGTGTTTGTGCCTGTCCTGGGAGA"\
        "GACCGGCGCACAGAGGAAGAGAATCTCCGCAAGAAAGGGGAGCCTCACCACGAGCTGCCC"\
        "CCAGGGAGCACTAAGCGAGCACTGCCCAACAACACCAGCTCCTCTCCCCAGCCAAAGAAG"\
        "AAACCACTGGATGGAGAATATTTCACCCTTCAGATCCGTGGGCGTGAGCGCTTCGAGATG"\
        "TTCCGAGAGCTGAATGAGGCCTTGGAACTCAAGGATGCCCAGGCTGGGAAGGAGCCAGGG"\
        "GGGAGCAGGGCTCACTCCAGCCACCTGAAGTCCAAAAAGGGTCAGTCTACCTCCCGCCAT"\
        "AAAAAACTCATGTTCAAGACAGAAGGGCCTGACTCAGACTGA" )


aminoacid_types = ['His', 'Asp', 'Trp', 'Ser', 'Cys', 
                   'Leu', 'Tyr', 'Met', 'Ter', 'Ile', 
                   'Gln', 'Gly', 'Val', 'Arg', 'Thr', 
                   'Asn', 'Phe', 'Lys', 'Ala', 'Pro', 'Glu']

ev_types = ['Tv', 'Ts', 'Td']

complexity_types = ['DMU', 'DMD', 'SM', 'MM', 'DMS']


#Labels selected for classification
target_labels = ['Colorectal carcinoma', 
                 'Lung (NSCLC)',
                 'Bladder carcinoma',
                 'Esophageal SCC',
                 'Hepatocellular carcinoma']
                 #'Breast carcinoma', 
                 #'Ovarian carcinoma']


#inputs and labels types
INPUTS = [ [], [], [], [], [], [], [], [] ]
LABELS = [ [], [], [], [] ]

# input indices 
SIMPLE_CODON = 0
CODON_EXTENDED = 1
SEQUENCE_FLAT = 2
SEQUENCE_EXTENDED = 3
SEQUENCE_2D = 4
SEQUENCE_FREQ_FLAT = 5
SEQUENCE_FREQ_2D = 6
PROTVEC = 7

#labels indices 
STD_LABELS_INT = 0
STD_LABELS_VECT = 1
BLACKLIST_LABELS_INT = 2
BLACKLIST_LABELS_VECT = 3

BALANCE = False
WINDOW_SIZE = 5

diseases = []
codons = []
positions = []

aminoacids = []
mutant_aminoacids = []
event_types = []
complexities = []


#contains sequences for each mutant codon
input_seqs = []

#includes most relevant subsequences
blacklist = []

#index {codon_id : list of subseq}
codon2subseq= {}

#inverted index
disease2subseq = {}


#Utilities

**Build inverted index** {disease : {subseq : freq}}

In [ ]:

#Given a seq builds all possible subsequences of length WINDOW_SIZE 
def create_subsequences(seq):
  result = []
  for i in range(0, len(seq) - WINDOW_SIZE * 3 + 1, 3):
    result.append(seq[i : i + WINDOW_SIZE * 3])
  
  return result


#Adds subsequences to key=disease in the inverted index
# each subsequence is a dict {subseq : frequency}
def add_subsequences(disease, subsequences):   
  if disease not in disease2subseq:
    disease2subseq[disease] = {}

  for s in subsequences:
    if s not in disease2subseq[disease]:
      disease2subseq[disease][s] = 0
    disease2subseq[disease][s] += 1
    

#Transforms integer labels to vector labels
def encode_labels(labels) :
  out = [[0] * len(target_labels) for x in labels]
  
  for i in range(len(labels)):
    label = labels[i]
    out[i][label] = 1

  return out


**Build sequence of adjacent codons for each input codon**

In [ ]:
offset_left = WINDOW_SIZE - 1
offset_right = len(TP53) // 3 - WINDOW_SIZE


#computes the required padding
def pad(pos):
  if pos < offset_left: 
    return WINDOW_SIZE - 1 - pos
  elif pos > offset_right:
    return WINDOW_SIZE - 1 - (len(TP53) // 3 - 1 - pos)
  else:
    return 0



#given args=mutant_codon, position builds a sequence of WINDOW_SIZE - 1 adjacent codons
def build_sequence(mutant_codon, pos):
  
  shift = 3 * (WINDOW_SIZE - 1)
  padding = "".join(['0'] * 3 * pad(pos))

  if pos < offset_left:
    seq = TP53[: 3 * (pos + WINDOW_SIZE)]
    pad_seq = padding + seq
  
  elif pos > offset_right:
    seq = TP53[3 * (pos - WINDOW_SIZE + 1) :]
    pad_seq = seq + padding
  
  else:
    seq = TP53[3 * (pos - (WINDOW_SIZE - 1)) : 3 * (pos + WINDOW_SIZE)]
    pad_seq = seq
  
  #replace codon with mutant codon in input sequence
  pad_seq, mutant_codon = list(pad_seq), list(mutant_codon)
  pad_seq[shift : shift + 3] = mutant_codon
  
  return "".join(pad_seq)


**Create blacklist of most relevant subsequences for all diseases**

In [ ]:
import math 

def last_valid(threshold):
    sum = 0
    for v in disease2subseq[x].values():
      if (sum + v) > threshold:
        break
      sum += v

    return v


#assumes all items key,value in inverted index are ordered by value 
def create_blacklist(percentage):    
  for x in disease2subseq:  
    threshold = percentage * math.fsum([v for v in disease2subseq[x].values()])                                
    min_freq = last_valid(threshold)                                                                  
    
      
    for k, v in disease2subseq[x].items():
      if v >= min_freq and v not in blacklist:
        blacklist.append(k)


#Preprocessing

**Load biovec pretrained model**

In [ ]:
import biovec

pv = biovec.models.load_protvec(biovec_model_path)

**Load data and populate structures**

In [ ]:
import csv


with open(dataset_path, 'r', newline="", encoding='utf8', errors='replace') as dataset:
  reader = csv.reader(dataset, delimiter="\t")

  codon_id = 0
  for row in reader:
    codon_pos, codon, mutant_codon = [row[i] for i in range(4, 7)]
    aa, mutant_aa, ev_type, complexity = row[7], row[8], row[12], row[13]
    disease = row[-1]
    
    if disease in target_labels:
      disease_id = target_labels.index(disease)
      diseases.append(disease_id)

      #INPUT TYPE 0, 1
      codons.append("".join([codon, mutant_codon]))
      positions.append(int(codon_pos))

      #OTHER INPUT TYPES
      # build codon sequence and all possible subseqs
      codon_seq = build_sequence(mutant_codon, int(codon_pos)-1)
      codon_subseqs = create_subsequences(codon_seq)


      add_subsequences(disease_id, subsequences=codon_subseqs)
      codon2subseq[codon_id] = codon_subseqs
      input_seqs.append(codon_seq)

      for item, l in zip([aa, mutant_aa, ev_type, complexity], [aminoacids, mutant_aminoacids, event_types, complexities]):
        l.append(item)

      codon_id += 1


**Encoding**

In [ ]:
base_encoder = {'A' : [0, 0, 0, 1], 
                'C' : [0, 0, 1, 0], 
                'G' : [0, 1, 0, 0], 
                'T' : [1, 0, 0, 0],
                '0' : [0, 0, 0, 0] }

evtype_encoder = { 'Tv' : [0, 0, 1],
                   'Ts' : [0, 1, 0], 
                   'Td' : [1, 0, 0] }

complexity_encoder = {'DMU' : [0, 0, 0, 0, 1], 
                      'DMD' : [0, 0, 0, 1, 0], 
                      'SM'  : [0, 0, 1, 0, 0], 
                      'MM'  : [0, 1, 0, 0, 0], 
                      'DMS' : [1, 0, 0, 0, 0] }

aminoacid_encoder = { 'His' : [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                      'Asp' : [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                      'Trp':  [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                      'Ser':  [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                      'Cys':  [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                      'Leu':  [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                      'Tyr':  [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                      'Met':  [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                      'Ter':  [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                      'Ile':  [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                      'Gln':  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                      'Gly':  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                      'Val':  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
                      'Arg':  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
                      'Thr':  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
                      'Asn':  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
                      'Phe':  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
                      'Lys':  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
                      'Ala':  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
                      'Pro':  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
                      'Glu':  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]}


protein_encoder = { "ATG" : "M",
                    "GAG" : "E",
                    "CCG" : "P",
                    "CAG" : "Q",
                    "TCA" : "S",
                    "GAT" : "D",
                    "CCT" : "P",
                    "AGC" : "S",
                    "GTC" : "V",
                    "CCC" : "P",
                    "CTG" : "L",
                    "AGT" : "S",
                    "GAA" : "E",
                    "ACA" : "T",
                    "TTT" : "F",
                    "GAC" : "D",
                    "CTA" : "L",
                    "TGG" : "W",
                    "AAA" : "K",
                    "CTT" : "L",
                    "AAC" : "N",
                    "GTT" : "V",
                    "TCC" : "S",
                    "TTG" : "L",
                    "CAA" : "Q",
                    "GCA" : "A",
                    "ATT" : "I",
                    "TTC" : "F",
                    "ACT" : "T",
                    "CCA" : "P",
                    "GGT" : "G",
                    "GCT" : "A",
                    "AGA" : "R",
                    "GAG" : "E",
                    "GTG" : "V",
                    "GCC" : "A",
                    "GCG" : "A",
                    "TCT" : "S",
                    "ACC" : "T",
                    "TAC" : "Y",
                    "GGC" : "G",
                    "CGT" : "R",
                    "CTG" : "L",
                    "CAT" : "H",
                    "GGG" : "G",
                    "AAG" : "K",
                    "TGC" : "C",
                    "ACG" : "T",
                    "CTC" : "L",
                    "CGC" : "R",
                    "ATC" : "I",
                    "CAC" : "H",
                    "AGG" : "R",
                    "CGA" : "R",
                    "GGA" : "G",
                    "AAT" : "N",
                    "TAT" : "Y",
                    "TGT" : "C",
                    "CGG" : "R",
                    "TGA" : "0",

                    "TAA" : "0",
                    "TAG" : "0",
                    "ATA" : "I",
                    "GTA" : "V",
                    "TTA" : "L",
                    "TCG" : "S",
                    "000" : "0"
                    }


stop_codons = ['TGA', 'TAA', 'TAG', '000']



#seq is an iterable 
def one_hot_encoding(seq, encoder):
  return [x for el in seq for x in encoder[el]]


#ProtVec encoder
def encode_protein(elem):
  result = ""
  for c in range(0, len(elem), 3):
    if elem[c:c+3] in stop_codons:
      continue
    result += "".join( one_hot_encoding( [elem[c:c+3]], protein_encoder) )
  return result



#Codons
codons = [one_hot_encoding(c, base_encoder) for c in codons]

#Aminoacids, Mutant aminoacids
aminoacids = [one_hot_encoding([a], aminoacid_encoder) for a in aminoacids]
mutant_aminoacids = [one_hot_encoding([m_a], aminoacid_encoder) for m_a in mutant_aminoacids]

#Event types
event_types = [one_hot_encoding([t], evtype_encoder) for t in event_types]

#Complexity
complexities = [one_hot_encoding([c], complexity_encoder) for c in complexities]


#binary representation of codon positions (array)
positions = [list(map(int, "{0:09b}".format(pos))) for pos in positions]


**Input type 0**: 

> *codon || mutant_codon*



**Input type 1**:

> *codon || mutant_codon || binary_position || aminoacid || mutant_aminoacid || type_of_event || complexity*



**Input type 2**: 

> *left_adjacent_codons || mutant_codon || right_adjacent_codons*



**Input type 3:**


> *left_adjacent_codons || mutant_codon || right_adjacent_codons || aminoacid || mutant_aminoacid || type_of_event || complexity*



**Input type 4** (2D):
> *left_adjacent_codons || mutant_codon || right_adjacent_codons*

**Input type 5:**


> *for i in codon_subsequences :  freq_i in j for all j in diseases =  freq_00 || freq_01 || ... || freq_0j|| ... ||freq_i0 || ... || freq_ij*



**Input 6** (2D):
> *for i in codon_subsequences :  freq_i in j for all j in diseases =  freq_00 || freq_01 || ... || freq_0j|| ... ||freq_i0 || ... || freq_ij*

**Input type 7**: (PROTVEC)

>*for i in codon_sequence: Embedding(i) where Embedding(i) = Word2Vec(amino acid)*



**Simple and extended inputs**

In [ ]:

# INPUT TYPE 0:     codon || mutant_codon
INPUTS[SIMPLE_CODON] = [c for c in codons]

# INPUT TYPE 1:     codon || mutant_codon || binary_position || aminoacid || mutant_aminoacid || type_of_event || complexity
INPUTS[CODON_EXTENDED] = [c + p + a + m_a + t + cx for c, p, a, m_a, t, cx in zip(codons, positions, aminoacids, mutant_aminoacids, event_types, complexities)]


#full set of labels
LABELS[STD_LABELS_INT] = diseases


**Sort inverted index and create blacklist**

In [ ]:
PERCENTAGE = 0.8

#For each disease sort subsequences by frequency in descending order
for x in disease2subseq:   
  disease2subseq[x] = dict(sorted(disease2subseq[x].items(), key=lambda x: x[1], reverse=True))  

create_blacklist(PERCENTAGE)

    

**Create sequence inputs and filter inputs using blacklist**

In [ ]:

# Adds to inputs the codon sequence if at least 1 among its WINDOW_SIZE-long subsequences is in blacklist 
for c_id in range(len(codon2subseq)):
  add = False
  for s in codon2subseq[c_id]:
    if s in blacklist:
      add = True
      break
  
  if add == True:

    #INPUT TYPE 2: SEQUENCE FLAT
    INPUTS[SEQUENCE_FLAT].append(one_hot_encoding(input_seqs[c_id], base_encoder))

    #INPUT TYPE 3: SEQUENCE EXTENDED
    INPUTS[SEQUENCE_EXTENDED].append(one_hot_encoding(input_seqs[c_id], base_encoder) + aminoacids[c_id] + mutant_aminoacids[c_id] + event_types[c_id] + complexities[c_id])

    #INPUT TYPE 5: SEQUENCE FREQUENCE      
    freqs = []
    for s in codon2subseq[c_id]: 
      freqs += [disease2subseq[k][s] if s in disease2subseq[k] else 0 for k in range(len(target_labels))]

    INPUTS[SEQUENCE_FREQ_FLAT].append(freqs)

    
    #INPUT TYPE 7: PROTVEC 
    INPUTS[PROTVEC].append(np.array(pv.to_vecs(encode_protein(input_seqs[c_id]))).flatten())

    #filtered labels
    LABELS[BLACKLIST_LABELS_INT].append(diseases[c_id])


  

**Parse inputs**

In [ ]:
#Label formats
LABELS[STD_LABELS_VECT] = encode_labels(LABELS[STD_LABELS_INT])
LABELS[BLACKLIST_LABELS_VECT] = encode_labels(LABELS[BLACKLIST_LABELS_INT])


#Label types
LABELS[STD_LABELS_INT] = np.array(LABELS[STD_LABELS_INT])
LABELS[STD_LABELS_VECT] = np.array(LABELS[STD_LABELS_VECT])
LABELS[BLACKLIST_LABELS_INT] = np.array(LABELS[BLACKLIST_LABELS_INT])
LABELS[BLACKLIST_LABELS_VECT] = np.array(LABELS[BLACKLIST_LABELS_VECT])


#Flat Input types
INPUTS[SIMPLE_CODON] = np.array(INPUTS[SIMPLE_CODON])
INPUTS[CODON_EXTENDED] = np.array(INPUTS[CODON_EXTENDED])
INPUTS[SEQUENCE_EXTENDED] = np.array(INPUTS[SEQUENCE_EXTENDED])
INPUTS[SEQUENCE_FLAT] = np.array(INPUTS[SEQUENCE_FLAT])
INPUTS[SEQUENCE_FREQ_FLAT] = np.array(INPUTS[SEQUENCE_FREQ_FLAT])
INPUTS[PROTVEC] = np.array(INPUTS[PROTVEC])


#2D input types
INPUTS[SEQUENCE_2D] = INPUTS[SEQUENCE_FLAT].reshape( (INPUTS[SEQUENCE_FLAT].shape[0], (WINDOW_SIZE*2-1)*3, 4 ) ) 
INPUTS[SEQUENCE_FREQ_2D] = INPUTS[SEQUENCE_FREQ_FLAT].reshape( (INPUTS[SEQUENCE_FREQ_FLAT].shape[0], len(target_labels), WINDOW_SIZE) )



#Split dataset

In [ ]:
#Deterministic split
SEED = 42


#Input type 0:
Xtrain, Xtest, ytrain, ytest = train_test_split(INPUTS[SIMPLE_CODON], LABELS[STD_LABELS_INT], 
                                                            train_size=0.7, random_state=SEED)
Simple_input = [Xtrain, Xtest, ytrain, ytest]


#Input type 1:
Xtrain, Xtest, ytrain, ytest, ytrain_v, ytest_v = train_test_split(INPUTS[CODON_EXTENDED], LABELS[STD_LABELS_INT], 
                                                                 LABELS[STD_LABELS_VECT], train_size=0.7, random_state=SEED)
Extended_input_int = [Xtrain, Xtest, ytrain, ytest]
Extended_input_vec = [Xtrain, Xtest, ytrain_v, ytest_v]


#Input type 2:
Xtrain, Xtest, ytrain, ytest, ytrain_v, ytest_v = train_test_split(INPUTS[SEQUENCE_FLAT], LABELS[BLACKLIST_LABELS_INT], 
                                                                 LABELS[BLACKLIST_LABELS_VECT], train_size=0.7, random_state=SEED)
subseqFlat_input_int = [Xtrain, Xtest, ytrain, ytest]
subseqFlat_input_vec = [Xtrain, Xtest, ytrain_v, ytest_v]


#Input type 3:
Xtrain, Xtest, ytrain, ytest, ytrain_v, ytest_v = train_test_split(INPUTS[SEQUENCE_EXTENDED], LABELS[BLACKLIST_LABELS_INT], 
                                                                 LABELS[BLACKLIST_LABELS_VECT], train_size=0.7, random_state=SEED)
subseqExtended_input_int = [Xtrain, Xtest, ytrain, ytest]
subseqExtended_input_vec = [Xtrain, Xtest, ytrain_v, ytest_v]


#Input type 4:
Xtrain, Xtest, ytrain, ytest, ytrain_v, ytest_v = train_test_split(INPUTS[SEQUENCE_2D], LABELS[BLACKLIST_LABELS_INT], 
                                                                 LABELS[BLACKLIST_LABELS_VECT], train_size=0.7, random_state=SEED)
subseq2D_input_int = [Xtrain, Xtest, ytrain, ytest]
subseq2D_input_vec = [Xtrain, Xtest, ytrain_v, ytest_v]


#Input type 5:
Xtrain, Xtest, ytrain, ytest, ytrain_v, ytest_v = train_test_split(INPUTS[SEQUENCE_FREQ_FLAT], LABELS[BLACKLIST_LABELS_INT], 
                                                                 LABELS[BLACKLIST_LABELS_VECT], train_size=0.7, random_state=SEED)
Freq_input_int = [Xtrain, Xtest, ytrain, ytest]
Freq_input_vec = [Xtrain, Xtest, ytrain_v, ytest_v]


#Input type 6:
Xtrain, Xtest, ytrain, ytest, ytrain_v, ytest_v = train_test_split(INPUTS[SEQUENCE_FREQ_2D], LABELS[BLACKLIST_LABELS_INT], 
                                                                 LABELS[BLACKLIST_LABELS_VECT], train_size=0.7, random_state=SEED)
freq2D_input_int = [Xtrain, Xtest, ytrain, ytest]
freq2D_input_vec = [Xtrain, Xtest, ytrain_v, ytest_v]


#Input type 7:
Xtrain, Xtest, ytrain, ytest, ytrain_v, ytest_v = train_test_split(INPUTS[PROTVEC], LABELS[BLACKLIST_LABELS_INT], 
                                                                 LABELS[BLACKLIST_LABELS_VECT], train_size=0.7, random_state=SEED)
protvec_input_int = [Xtrain, Xtest, ytrain, ytest]
protvec_input_vec = [Xtrain, Xtest, ytrain_v, ytest_v]


#Balancing (Up-sampling)

In [ ]:
from imblearn.over_sampling import SMOTE

BALANCE = True

oversample = SMOTE()
inputs = [ Simple_input, 
           Extended_input_int, Extended_input_vec,
           subseqFlat_input_int, subseqFlat_input_vec, 
           subseqExtended_input_int, subseqExtended_input_vec, 
           Freq_input_int, Freq_input_vec,
           protvec_input_int, protvec_input_vec ]



for input in inputs:
  input[0], input[2] = oversample.fit_resample(input[0], input[2])


#Random forest classifier

In [ ]:
#Select Input
selected_input = Freq_input_int
Xtrain, Xtest, ytrain, ytest = [i for i in selected_input]


parameters = {'bootstrap': True,
              'min_samples_leaf': 6,
              'n_estimators': 100, 
              'min_samples_split': 6,
              'max_features': 'sqrt',
              'max_leaf_nodes': None}


#Model
RF_model = RandomForestClassifier(**parameters)

#Training
RF_model.fit(Xtrain, ytrain)

#Eval
RF_predictions = RF_model.predict(Xtest)

#Metrics 
ForestAccuracy = accuracy_score(ytest, RF_predictions)
ForestRecall = recall_score(ytest, RF_predictions, average='macro')
Forestprecision = precision_score(ytest, RF_predictions, average='macro')
F1 = 2 * (Forestprecision*ForestRecall/(Forestprecision+ForestRecall))

#Results
Forest_result_string = '\nRandom forest performance : \n\taccuracy = {:.5f}\n\trecall score = {:.5f}\n\tprecision = {:.5f}\n\tF1 = {:.5f}'
print(Forest_result_string.format(ForestAccuracy,ForestRecall,Forestprecision, F1))



Random forest performance : 
	accuracy = 0.50357
	recall score = 0.47409
	precision = 0.45221
	F1 = 0.46289


# SVM

In [ ]:
#Select Input
selected_input = Freq_input_int
Xtrain, Xtest, ytrain, ytest = [i for i in selected_input]

#Model
clf = SVC(C= 0.8, kernel='rbf', gamma='auto', max_iter=8000, verbose=False)

#Training
print('Fitting data into SVM. . .')
clf.fit(Xtrain, ytrain)

#Eval
print('predicting results from SVM. . .')
Result = clf.predict(Xtest)


#Metrics
SVMaccuracy = accuracy_score(ytest, Result)
SVMRecallScore = recall_score(ytest, Result, average='macro')
SVMprecision = precision_score(ytest, Result, average='macro')
F1 = 2 * (SVMprecision*SVMRecallScore/(SVMprecision+SVMRecallScore))


#Results
SVM_result_string = '\nSVM performance : \n\taccuracy = {:.5f}\n\trecall score = {:.5f}\n\tprecision = {:.5f}\n\tF1 = {:.5f}'
print(SVM_result_string.format(SVMaccuracy,SVMRecallScore,SVMprecision, F1))


Fitting data into SVM. . .
predicting results from SVM. . .

SVM performance : 
	accuracy = 0.50508
	recall score = 0.48911
	precision = 0.46088
	F1 = 0.47458


#Neural network models

In [ ]:

#Feed Forward Model
def FFW(input_shape, num_classes, opt, met) :
    model = keras.Sequential()

    model.add(Dense(64, activation='relu', input_shape=input_shape))
    model.add(BatchNormalization())
  
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.2))
    model.add(BatchNormalization())

    model.add(Dense(32, activation='relu'))
    model.add(BatchNormalization())

    model.add(Dense(16, activation='relu'))
    model.add(BatchNormalization())

    model.add(Dense(num_classes, activation='softmax'))

    # Compile
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=met)

    return model
    

#CNN Model
def CNN(input_shape, num_classes, opt, met) :
    model = keras.Sequential()

    model.add(Conv1D(64, kernel_size=1, activation='sigmoid',
                     strides=1, input_shape=input_shape))

    model.add(Conv1D(128, kernel_size=2, activation='relu', strides=2))
    model.add(MaxPooling1D(pool_size=2, strides=1))
    model.add(BatchNormalization())
    
    model.add(Flatten())

    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.3))
    model.add(BatchNormalization())
  
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.2))
    model.add(BatchNormalization())

    model.add(Dense(32, activation='relu'))
    model.add(Dropout(0.2))
    model.add(BatchNormalization())

    model.add(Dense(num_classes, activation='softmax'))
    
    # Compile
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=met)
    
    return model



#LSTM Model
def LSTM_net(input_shape, num_classes, opt, met) :
    model = keras.Sequential()

    model.add(Bidirectional(LSTM(64, return_sequences=True,
                                 dropout=0.1, recurrent_dropout=0.2), 
                            input_shape=input_shape))

    model.add(Bidirectional(LSTM(32)))

    model.add(Dense(32, activation='relu'))
    model.add(Dropout(0.2))
    model.add(BatchNormalization())

    model.add(Dense(16, activation='relu'))
    model.add(Dropout(0.2))
    model.add(BatchNormalization())

    model.add(Dense(num_classes, activation='softmax'))
    
    # Compile
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=met)
    
    return model

#FFW Training and evaluation

In [ ]:
#Opt and metrics
optimizer = Adamax(learning_rate=0.002, beta_1=0.9, beta_2=0.9, epsilon=1e-06, name="Adamax")
metrics = [CategoricalAccuracy(), Precision(), Recall()]


#Select Input 
selected_input = Freq_input_vec
Xtrain, Xtest, ytrain, ytest = [i for i in selected_input]


#FFW training
ffw = FFW(Xtrain[0].shape, len(target_labels), optimizer, metrics)
ffw.summary()
ffw.fit(Xtrain, ytrain, batch_size=32, epochs=15, validation_split=0.2)
print('\n')


#FFW eval
score = ffw.evaluate(Xtest, ytest)


#Metrics
loss, accuracy, precision, recall = [x for x in score]
F1 = 2 * (precision*recall/(precision+recall))


#Results
res = "loss : {:.5f}\naccuracy : {:.5f}\nprecision : {:.5f}\nrecall : {:.5f}\nF1 : {:.5f}\n"
print("FFW results:")
print(res.format(loss,accuracy,precision,recall,F1))



Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 64)                1664      
_________________________________________________________________
batch_normalization (BatchNo (None, 64)                256       
_________________________________________________________________
dense_1 (Dense)              (None, 64)                4160      
_________________________________________________________________
dropout (Dropout)            (None, 64)                0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 64)                256       
_________________________________________________________________
dense_2 (Dense)              (None, 32)                2080      
_________________________________________________________________
batch_normalization_2 (Batch (None, 32)                1

#CNN Training and Evaluation

In [ ]:
#Opt and metrics
optimizer = Adamax(learning_rate=0.002, beta_1=0.9, beta_2=0.9, epsilon=1e-06, name="Adamax")
metrics = [CategoricalAccuracy(), Precision(), Recall()]


#Select Input 
is_Frequence = False
selected_input = subseq2D_input_vec
Xtrain, Xtest, ytrain, ytest = [i for i in selected_input]


#Check for balanced input
if BALANCE :
  #Check input type
  if is_Frequence :
    Xtrain = Freq_input_vec[0].reshape( (Freq_input_vec[0].shape[0], len(target_labels), WINDOW_SIZE) )
    ytrain = Freq_input_vec[2]
  else : 
    Xtrain = subseqFlat_input_vec[0].reshape( (subseqFlat_input_vec[0].shape[0], (WINDOW_SIZE*2-1)*3, 4) )
    ytrain = subseqFlat_input_vec[2]


#CNN training
cnn = CNN(Xtrain[0].shape, len(target_labels), optimizer, metrics)
cnn.summary()
cnn.fit(x=Xtrain, y=ytrain, batch_size=32, epochs=20, validation_split=0.2)
print('\n')


#CNN eval
score = cnn.evaluate(Xtest, ytest)


#Metrics
loss, accuracy, precision, recall = [x for x in score]
F1 = 2 * (precision*recall/(precision+recall))


#Results
res = "loss : {:.5f}\naccuracy : {:.5f}\nprecision : {:.5f}\nrecall : {:.5f}\nF1 : {:.5f}\n"
print("CNN results:")
print(res.format(loss,accuracy,precision,recall,F1))



Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_6 (Conv1D)            (None, 27, 64)            320       
_________________________________________________________________
conv1d_7 (Conv1D)            (None, 13, 128)           16512     
_________________________________________________________________
max_pooling1d_3 (MaxPooling1 (None, 12, 128)           0         
_________________________________________________________________
batch_normalization_14 (Batc (None, 12, 128)           512       
_________________________________________________________________
flatten_3 (Flatten)          (None, 1536)              0         
_________________________________________________________________
dense_15 (Dense)             (None, 64)                98368     
_________________________________________________________________
dropout_11 (Dropout)         (None, 64)               

#LSTM Training and evaluation

In [ ]:
#Opt and metrics
optimizer = Adamax(learning_rate=0.002, beta_1=0.9, beta_2=0.9, epsilon=1e-06, name="Adamax")
metrics = [CategoricalAccuracy(), Precision(), Recall()]


#Select Input 
is_Frequence = False
selected_input = subseq2D_input_vec
Xtrain, Xtest, ytrain, ytest = [i for i in selected_input]

#Check for balanced input
if BALANCE :
  #Check input type
  if is_Frequence :
    Xtrain = Freq_input_vec[0].reshape( (Freq_input_vec[0].shape[0], len(target_labels), WINDOW_SIZE) )
    ytrain = Freq_input_vec[2]
  else : 
    Xtrain = subseqFlat_input_vec[0].reshape( (subseqFlat_input_vec[0].shape[0], (WINDOW_SIZE*2-1)*3, 4) )
    ytrain = subseqFlat_input_vec[2]


#LSTM training
lstm = LSTM_net(Xtrain[0].shape, len(target_labels), optimizer, metrics)
lstm.summary()
lstm.fit(x=Xtrain, y=ytrain, batch_size=32, epochs=8, validation_split=0.2)
print('\n')


#LSTM eval
score = lstm.evaluate(Xtest, ytest)


#Metrics
loss, accuracy, precision, recall = [x for x in score]
F1 = 2 * (precision*recall/(precision+recall))


#Results
res = "loss : {:.5f}\naccuracy : {:.5f}\nprecision : {:.5f}\nrecall : {:.5f}\nF1 : {:.5f}\n"
print("LSTM results:")
print(res.format(loss,accuracy,precision,recall,F1))


Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_2 (Bidirection (None, 27, 128)           35328     
_________________________________________________________________
bidirectional_3 (Bidirection (None, 64)                41216     
_________________________________________________________________
dense_19 (Dense)             (None, 32)                2080      
_________________________________________________________________
dropout_14 (Dropout)         (None, 32)                0         
_________________________________________________________________
batch_normalization_18 (Batc (None, 32)                128       
_________________________________________________________________
dense_20 (Dense)             (None, 16)                528       
_________________________________________________________________
dropout_15 (Dropout)         (None, 16)               

# Results




In [ ]:
# Classifier  input_type  accuracy   recall   precision  F1-score  [loss]

With UpSampling

In [ ]:
'''
RF	Simple_input	0.46582163875056587	0.4161661133443406	0.41667292731903266	0.41641936612397523
RF	Extended_input_int	0.47457371359589556	0.4330565024830971	0.4202551125665471	0.42655978423104207
RF	subseqFlat_input_int	0.46425861283958114	0.4338691125152671	0.41329839283445996	0.4233340060158415
RF	subseqExtended_input_int	0.464865685232964	0.4309110083678217	0.4090529954335421	0.4196976011839411
RF	Freq_input_int	0.5005311883442101	0.48023657650269946	0.4540078089999061	0.4667540084435397
RF	protvec_input_int	0.478828350280771	0.4368726802793647	0.4208707362795727	0.4287224432388813

SVM	Simple_input	0.4798551380715256	0.4018004263829374	0.41424219334486584	0.4079264633685148
SVM	Extended_input_int	0.49781198128866755	0.40308113419111125	0.44701338992802303	0.423912068831418
SVM	subseqFlat_input_int	0.4537866140537259	0.40233009629640987	0.4073951956784375	0.4048468040518906
SVM	subseqExtended_input_int	0.488845044771589	0.40400492977439645	0.4372510247745522	0.4199710411619971
SVM	Freq_input_int	0.5015935650326302	0.4884856849196019	0.46041166552668306	0.47403337710659527
SVM	protvec_input_int	0.4231294581878889	0.3962951103992245	0.4283905822493067	0.4117182936477355

FFW	Extended_input_vec	0.5124490857124329	0.3260902464389801	0.6289289593696594	0.4294941884018145	1.3252984285354614
FFW	subseqFlat_input_vec	0.49810290336608887	0.31658825278282166	0.625674843788147	0.42043736261326753	1.313447117805481
FFW	subseqExtended_input_vec	0.5199574828147888	0.3300956189632416	0.6306175589561462	0.4333532593016734	1.2918874025344849
FFW	Freq_input_vec	0.5481863617897034	0.2177872210741043	0.6846374273300171	0.3304548097289039	1.1757991313934326
FFW	protvec_input_vec	0.49491578340530396	0.32888147234916687	0.6032850742340088	0.4256949236971003	1.3194283246994019

CNN	subseq2D_input_vec	0.52102	0.28259	0.67293 0.39803	1.29374
CNN	freq2D_input_vec	0.5625	0.2879  0.6921	0.40665	1.1750

LSTM	subseq2D_input_vec	0.49476	0.15420	0.68418	0.25167	1.34640
LSTM	freq2D_input_vec	0.54287	0.39475 0.70297	0.50559	1.17333

'''

Unbalanced

In [ ]:
'''
RF	subseqExtended_input_int	0.5430262558810138	0.3958388791192903	0.5256732349197897	0.4516096987192432
RF	Freq_input_int	0.579147063287297	0.450222225124305	0.542562300633899	0.49209793247613487

SVM	subseqExtended_input_int	0.5296706632265897	0.3691305253042242	0.4528985484849394	0.40674638998144436
SVM	Freq_input_int	0.5859766277128547	0.45120391445178887	0.5712248625168528	0.5041698743338361

FFW	subseqExtended_input_vec	0.5402944087982178	0.38716042041778564	0.6375905871391296	0.48177525652737263	1.2360925674438477
FFW	Freq_input_vec	0.5941721200942993	0.41205039620399475	0.7195865511894226	0.524030121504307	1.0828877687454224

CNN	subseq2D_input_vec	0.5352860689163208	0.3593868613243103	0.6473482847213745	0.4621840593198747	1.2313532829284668
CNN	freq2D_input_vec	0.5944756269454956	0.45940202474594116	0.684377133846283	0.5497638921236833	1.0882291793823242

LSTM	subseq2D_input_vec	0.48808619379997253	0.27075427770614624	0.6209536790847778	0.3770872819720573	1.2995712757110596
LSTM	freq2D_input_vec	0.5987251400947571	0.4962816834449768	0.6725627183914185	0.5711291555680356	1.0860388278961182
'''

With Breast and Ovarian Carcinoma

In [ ]:
'''
RF	Freq_input_int	0.4249859943977591	0.41214062843062776	0.3970174766483394	0.40443772680944734
SVM	Freq_input_int	0.4355182072829132	0.42481942285783153	0.411057909296946	0.4178253843503929
FFW	Freq_input_vec	0.4628571569919586	0.2010084092617035	0.6476534008979797	0.30679778047969425	1.2878766059875488
CNN	freq2D_input_vec	0.473613440990448	0.19450980424880981	0.6531226634979248	0.29974963503970153	1.2739077806472778
LSTM	freq2D_input_vec	0.4778711497783661	0.17579832673072815	0.6866520643234253	0.27992864333150713	1.2774616479873657
'''

'\nRF\tFreq_input_int\t0.4249859943977591\t0.41214062843062776\t0.3970174766483394\t0.40443772680944734\nSVM\tFreq_input_int\t0.4355182072829132\t0.42481942285783153\t0.411057909296946\t0.4178253843503929\nFFW\tFreq_input_vec\t0.4628571569919586\t0.2010084092617035\t0.6476534008979797\t0.30679778047969425\t1.2878766059875488\nCNN\tfreq2D_input_vec\t0.473613440990448\t0.19450980424880981\t0.6531226634979248\t0.29974963503970153\t1.2739077806472778\nLSTM\tfreq2D_input_vec\t0.4778711497783661\t0.17579832673072815\t0.6866520643234253\t0.27992864333150713\t1.2774616479873657\n'